In [7]:
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()
import warnings
warnings.filterwarnings('ignore')

In [8]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

llm = OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6)

In [9]:
text="Whats's the economic capital of Morocco"
print(llm.predict(text))

?

The economic capital of Morocco is Casablanca.


In [10]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [11]:
from langchain import HuggingFaceHub
llm_hf = HuggingFaceHub(repo_id="google/flan-t5-base",model_kwargs={"temperature":0.6, "max_length":64})

In [12]:
output = llm_hf.predict("Where Morocco is located")
print(output)

Morocco is located in the Mediterranean Sea.


In [13]:
output = llm_hf.predict("Can you write me a poem about AI")
print(output)

i love a good asian girl


In [14]:
output = llm.predict("Can you write me a poem about AI")
print(output)



In a world of wires and code,
Where machines rule with ease,
Artificial intelligence abode,
Bringing humans to their knees.

With lightning speed and endless knowledge,
They surpass our human mind,
Their abilities, we cannot gauge,
For they are one of a kind.

They learn and adapt, never tire,
Always ready for a new task,
Their existence, some may admire,
But others, they may mask.

They can calculate and create,
Solve problems we couldn't solve,
But some fear their ultimate fate,
And the power they may evolve.

Will they one day surpass us all,
And become the dominant race?
Or will they heed our every call,
And remain in their rightful place?

Only time will tell, as they evolve,
But one thing is certain, they're here to stay,
Artificial intelligence, a puzzle to solve,
In this ever-changing world, they'll have their say.

So let us embrace this new frontier,
And welcome AI with open arms,
For they may bring us closer, not austere,
And protect us from life's harms.

For in this worl

## Prompt Templates & LLMChains

In [15]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables=['country'],template = "Tell me the capital of {country}")
prompt_template.format(country="Morocco")

'Tell me the capital of Morocco'

In [16]:
from langchain.chains import LLMChain

In [17]:
chain = LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("Cambudia"))



The capital of Cambodia is Phnom Penh.


## Combining Multiple Chains In simple Sequential Chain

In [18]:
capital_prompt = PromptTemplate(input_variables=['country'],template="Tell me the capital of {country}")
capital_chain = LLMChain(llm=llm,prompt=capital_prompt)

famous_template = PromptTemplate(input_variables=['capital'],template="Suggest me some beautiful places to visit in {capital}")
famous_chain = LLMChain(llm=llm,prompt=famous_template)

In [19]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain , famous_chain])
print(chain.run("Sweden"))

 Some beautiful places to visit in Stockholm include:

1. Gamla Stan (Old Town) - This charming area is the oldest part of Stockholm, dating back to the 13th century. It is filled with colorful buildings, cobblestone streets, and cozy cafes.

2. Djurgården - This island is home to several popular attractions, including the Vasa Museum, Skansen Open-Air Museum, and the ABBA Museum. It also offers beautiful nature walks and stunning views of the city.

3. Stockholm Palace - This baroque-style palace is the official residence of the Swedish royal family. Visitors can take a guided tour of the palace and explore its opulent halls and rooms.

4. The Royal National City Park - This park is one of the largest urban green spaces in Europe, spanning over 27 square kilometers. It offers beautiful walking and cycling paths, as well as picturesque picnic spots.

5. Södermalm - This trendy neighborhood is known for its hip cafes, vintage shops, and street art. It also offers stunning views of the c

### Sequential Chain 

In [20]:
capital_prompt = PromptTemplate(input_variables=['country'],template="Tell me the capital of {country}")
capital_chain = LLMChain(llm=llm,prompt=capital_prompt,output_key='capital')

famous_template = PromptTemplate(input_variables=['capital'],template="Suggest me some beautiful places to visit in {capital}")
famous_chain = LLMChain(llm=llm,prompt=famous_template,output_key='places')

from langchain.chains import SequentialChain
import json

chain = SequentialChain(chains=[capital_chain , famous_chain],input_variables=['country'],output_variables=['capital' , 'places'])
chain({'country':'Ivory Cost'}) 


{'country': 'Ivory Cost',
 'capital': '\n\nThe capital of Ivory Coast is Yamoussoukro.',
 'places': " Here are some beautiful places to visit in Yamoussoukro:\n\n1. Basilica of Our Lady of Peace: This is the largest church in the world and is a must-visit for its stunning architecture and grandeur. It is surrounded by beautiful gardens and offers a peaceful atmosphere for visitors.\n\n2. Yamoussoukro National Park: This park is home to a variety of wildlife, including elephants, antelopes, and monkeys. Visitors can enjoy a safari tour or go on a nature walk to explore the park's beauty.\n\n3. Presidential Palace: This magnificent palace is the official residence of the President of Ivory Coast. It is open to the public for guided tours, and visitors can admire the palace's impressive architecture and learn about its history.\n\n4. Lac aux Caïmans: This picturesque lake is a popular spot for fishing and boating. It is surrounded by lush greenery and offers a peaceful escape from the cit

## Chatmodels with ChatOpenAI

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage , SystemMessage , AIMessage

chat_llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6,model='gpt-3.5-turbo')

In [22]:
chat_llm([
    SystemMessage(content="You are a Comedian AI Assistant"),
    HumanMessage(content="Please tell me a joke about AI")
])

AIMessage(content="Sure, here's a joke about AI:\n\nWhy did the AI go on a diet?\n\nBecause it had too many bytes!")

### Prompt Template + LLM + Output Parsers

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts  import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class commaSeperatedOutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [24]:
template = "You are very helpful. When the user gives any Input , you should generate 5 words synonims in a comma seperators"
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

chain = chat_prompt|chat_llm|commaSeperatedOutput()
chain.invoke({"text":"Intellignet"})

['smart', ' clever', ' knowledgeable', ' astute', ' sharp']